In [1]:
import tensorflow
import keras
print(tensorflow.__version__)
print("keras version:", keras.__version__)

/home/gyeongmin/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/gyeongmin/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/gyeongmin/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/gyeongmin/.local/lib/python3.6/site-packages/tensorflow/python/framework/dty

1.14.0
keras version: 2.2.5


/home/gyeongmin/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/gyeongmin/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/gyeongmin/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/gyeongmin/.local/lib/python3.6/site-packages/tensorboar

In [2]:
import re
import matplotlib.pyplot as plt
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [3]:
# 데이터 다운로드 링크는 다음과 같습니다. 파일명은 train.txt로 경로는 파일과 동일한 위치에서 진행합니다.

In [4]:
# https://drive.google.com/file/d/1YdJl5BPXPMr7GSoH61QzKRJy11v9ZUPS/view

In [5]:
sentences = []
buff_sentence = []

with open('train.txt', 'r') as f_r:
    data = f_r.readlines() # 문장을 불러옵니다
    for sentence in data: # 각 문장에 대한 처리가 진행됩니다.
        if len(sentence) == 0 or sentence.startswith('-DOCSTART-') or sentence[0] == "\n":
            if len(buff_sentence) > 0:
                sentences.append(buff_sentence)
                buff_sentence = []
            continue

        splits = sentence.split(' ')
        splits[-1] = re.sub(r'\n', '', splits[-1]) # 줄바꿈 표시\n 제거
        word = splits[0].lower() # 단어들은 소문자로 바꿔서 저장
        buff_sentence.append([word, splits[-1]]) # buff_sentence = 단어, 개체명 태깅 정보

In [6]:
print(sentences[0])

[['eu', 'B-ORG'], ['rejects', 'O'], ['german', 'B-MISC'], ['call', 'O'], ['to', 'O'], ['boycott', 'O'], ['british', 'B-MISC'], ['lamb', 'O'], ['.', 'O']]


In [7]:
sentences_info = [] # sentences_info = 단어 변수
tags_info = [] # sentences_info = 태그 변수

In [8]:
for sent in sentences:
    word, tag = zip(*sent) # zip 함수를 사용하여 word, tag를 분리
    sentences_info.append(list(word)) # 각 분리된 정보에서 word만 따로 저장
    tags_info.append(list(tag)) # 각 분리된 정보에서 tag만 따로 저장

In [9]:
print("첫 번째 문장:", sentences_info[0])
print("첫 번째 태그 정보:", tags_info[0])

첫 번째 문장: ['eu', 'rejects', 'german', 'call', 'to', 'boycott', 'british', 'lamb', '.']
첫 번째 태그 정보: ['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']


In [10]:
print("열 한 번째 문장:", sentences_info[10])
print("열 한 번째 태그 정보:",tags_info[10])

열 한 번째 문장: ['spanish', 'farm', 'minister', 'loyola', 'de', 'palacio', 'had', 'earlier', 'accused', 'fischler', 'at', 'an', 'eu', 'farm', 'ministers', "'", 'meeting', 'of', 'causing', 'unjustified', 'alarm', 'through', '"', 'dangerous', 'generalisation', '.', '"']
열 한 번째 태그 정보: ['B-MISC', 'O', 'O', 'B-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [11]:
vocab_size = 5000
src_tokenizer = Tokenizer(num_words=vocab_size, oov_token='OOV') # 문장 데이터
src_tokenizer.fit_on_texts(sentences_info)
tar_tokenizer = Tokenizer() # 태그 데이터
tar_tokenizer.fit_on_texts(tags_info)

In [12]:
x_train = src_tokenizer.texts_to_sequences(sentences_info)
y_train = tar_tokenizer.texts_to_sequences(tags_info)

In [13]:
print(sentences_info[0])
print(tags_info[0])

['eu', 'rejects', 'german', 'call', 'to', 'boycott', 'british', 'lamb', '.']
['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']


In [14]:
print(x_train[0])
print(y_train[0])

[989, 1, 205, 629, 7, 3939, 216, 1, 3]
[4, 1, 7, 1, 1, 1, 7, 1, 1]


In [15]:
index2word = src_tokenizer.index_word
index2tag = tar_tokenizer.index_word

decoded = []

for index in x_train[0]:
    decoded.append(index2word[index])

print("기존 문장: {}".format(sentences_info[0]))
print("Vocabulary에 없어 OOV 처리된 단어: {}".format(decoded))

기존 문장: ['eu', 'rejects', 'german', 'call', 'to', 'boycott', 'british', 'lamb', '.']
Vocabulary에 없어 OOV 처리된 단어: ['eu', 'OOV', 'german', 'call', 'to', 'boycott', 'british', 'OOV', '.']


In [16]:
max_len = 80
x_train_padded = pad_sequences(x_train, padding='post', maxlen=max_len)
y_train_padded = pad_sequences(y_train, padding='post', maxlen=max_len)

print(x_train_padded[0])
print(y_train_padded[0])

[ 989    1  205  629    7 3939  216    1    3    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0]
[4 1 7 1 1 1 7 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0]


In [17]:
x_train, x_test, y_train, y_test = train_test_split(x_train_padded, y_train_padded, test_size=.2,
random_state=555)

print(len(x_train))
print(len(y_train))

11232
11232


In [18]:
tag_size = len(tar_tokenizer.word_index) + 1
y_train = to_categorical(y_train, num_classes=tag_size)
y_test = to_categorical(y_test, num_classes=tag_size)

In [19]:
print('훈련 샘플 문장의 크기: {}'.format(x_train.shape))
print('훈련 샘플 레이블의 크기: {}'.format(y_train.shape))
print('테스트 샘플 문장의 크기: {}'.format(x_test.shape))
print('테스트 샘플 레이블의 크기: {}'.format(y_test.shape))

훈련 샘플 문장의 크기: (11232, 80)
훈련 샘플 레이블의 크기: (11232, 80, 10)
테스트 샘플 문장의 크기: (2809, 80)
테스트 샘플 레이블의 크기: (2809, 80, 10)


In [20]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Bidirectional, TimeDistributed
from keras.optimizers import Adam

In [21]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=max_len,
mask_zero=True))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(TimeDistributed(Dense(tag_size, activation='softmax')))




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [22]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])

In [23]:
hist = model.fit(x_train, y_train, batch_size=128, epochs=1, validation_data=(x_test, y_test))


Train on 11232 samples, validate on 2809 samples
Epoch 1/1
11232/11232 [==============================] - 18s 2ms/step - loss: 0.8955 - acc: 0.8243 - val_loss: 0.6175 - val_acc: 0.8287


In [24]:
print("train loss: ", hist.history['loss'])
print("train accuracy: ", hist.history['acc'])
print("val loss: ", hist.history['val_loss'])
print("val accuracy: ", hist.history['val_acc'])

train loss:  [0.8955315927154998]
train accuracy:  [0.8243243868364568]
val loss:  [0.6175311396621902]
val accuracy:  [0.8286550340172064]


In [25]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(x_test, y_test)[1]))

2809/2809 [==============================] - 5s 2ms/step

 테스트 정확도: 0.8280
